# Mixture Density Networks

本文主要参考了以下资料简单了解 在不确定性量化方面的神经网络模型Mixture Density Networks

- [A Hitchhiker’s Guide to Mixture Density Networks](https://towardsdatascience.com/a-hitchhikers-guide-to-mixture-density-networks-76b435826cca)
- [Mixture Density Networks: Probabilistic Regression for Uncertainty Estimation](https://deep-and-shallow.com/2021/03/20/mixture-density-networks-probabilistic-regression-for-uncertainty-estimation/)
- [Mixture Density Networks](https://cedar.buffalo.edu/~srihari/CSE574/Chap5/Chap5.6-MixDensityNetworks.pdf)

## 介绍

不确定性是我们做出的每一个决定的关键因素。然而，在企业中，管理人员经常面临会产生各种不可预见后果的决策。经理可能会处理以下问题：

- “我们如何根据产品属性为 XYZ 设定价格？”
- “广告会产生多少网站访问量？”
- “客户 Z-A137 的预期客户终身价值是多少？”
- “鉴于市场情况，我们应该在广告上花多少钱？”

这每一个问题都意味着一个决定，理想情况下，该决定充分了解并考虑（或至少承认）潜在的不确定性。例如：定价错误会导致收入减少，进而可能导致需要节省成本。

本文探讨一些关于预测连续值的技术问题，以及众所周知的线性模型在某些情况下是如何受限的。然后，将展示一种称为混合密度网络 (MDN) 的神经网络变体，以规避这些限制。这样可能会更好地掌握预测的潜在不确定性。

前面提出的问题需要预测并具有三个基本特征。第一个是，我们希望预测连续量（价格，网站访问量，销售量，你的名字）。其次，预测是在不确定性下做出的。因此，理想情况下，我们希望了解我们对预测的不确定性。第三，预测以一些输入观察为条件（产品属性、客户、广告设计）。理想情况下，我们不希望获得问题的单一答案，而是希望获得一系列答案来评估每个单独答案的概率。简而言之：我们正在寻找给定输入的一系列答案的概率分布。因此，我们增强了对预测的理解，甚至可以做出更明智的决定。

## 高斯混合Gaussian Mixture的直觉

举个例子：让我们假设，我们将要预测产品的价格，比如耳机。查看市场价格的直方图，我们推断存在低价（~30 美元）、中价（~60 美元）和高价（~120 美元）的耳机。简单的高斯分布来对数据进行建模注定会失败，这很容易从经验高斯分布的曲线中看出。请记住，高斯正态分布 (𝒩) 由两个值参数化：均值 (μ) 和标准差 (σ)。根据样本均值 (μ=47.95) 和标准值将 𝒩 拟合到数据中。偏差 (σ=27.76) 将概率分配给我们在虚构市场中没有观察到的价格区域（即价格在 90 美元到 110 美元之间）。此外，根据曲线的左侧，负价格是“合理的”。

![](img/1_Af3tuCrG1eDYEPZ4Hm8oGA.png)

生成价格分布的基础数据是高斯分布的混合。混合物是多峰的；因此，它表现出多个“峰值”。为了适应我们虚构定价数据的分布，我们可以使用不是一个，不是两个，而是三个充分混合的高斯分量。我们“只需要”选择参数（μ，σ），通过加权（⍺）对分布进行归一化，然后对它们求和（稍后会详细介绍）。混合这些组件巧妙地反映了真实数据，如下图所示。

![](img/1_CBEfTShzie89SRPdG7YoMQ.png)